# Creating Redshift cluster using the AWS python SDK

In [1]:
import pandas as pd
import boto3
import json
import configparser
from botocore.exceptions import ClientError

In [2]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

KEY                   = config.get('AWS', 'KEY')
SECRET                = config.get('AWS', 'SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,redshift-cluster-1
4,DWH_DB,dev
5,DWH_DB_USER,awsuser
6,DWH_DB_PASSWORD,9507024922
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,myRedshiftRole


### Create clients for IAM, EC2, S3 and Redshift

#### Note:This resources are being created in the us-east-1 region

In [3]:
ec2 = boto3.resource('ec2', 
                    region_name = "us-west-2",
                   aws_access_key_id = KEY,
                   aws_secret_access_key=SECRET)
s3 = boto3.resource('s3',
                    region_name = "us-west-2",
                   aws_access_key_id = KEY,
                   aws_secret_access_key=SECRET)
iam = boto3.client('iam',
                   region_name = "us-west-2",
                   aws_access_key_id = KEY,
                   aws_secret_access_key=SECRET)
redshift = boto3.client('redshift',
                        region_name = "us-west-2",
                   aws_access_key_id = KEY,
                   aws_secret_access_key=SECRET)

In [4]:
s3bucket = s3.Bucket("udacity-dend")

s3_logs = iter(s3bucket.objects.filter(Prefix = "log-data"))
for _ in range(10):
    print(next(s3_logs))

s3_songs = iter(s3bucket.objects.filter(Prefix = "song_data"))
for _ in range(10):
    print(next(s3_songs))

s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-01-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-02-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-03-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-04-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-05-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-06-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-07-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-08-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-09-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/')
s3.ObjectSummary(bucket_name='udacity-dend', ke

In [5]:
#1.1 Create the Redshift Role

try:
    print("1.1 Creating a IAM Role : myRedshiftRole")
    dwhRole = iam.create_role(
        Path = '/',
        RoleName = DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
                {'Statement': [{"Action": "sts:AssumeRole",
                    'Effect': 'Allow',
                    'Principal': {'Service': 'redshift.amazonaws,com'}}],
                 'Version': '2022-08-25'
                    
                })
            )
except Exception as e:
    print(e)

print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName = DWH_IAM_ROLE_NAME,
                       PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                        )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName = DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.1 Creating a IAM Role : myRedshiftRole
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name myRedshiftRole already exists.
1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::338405110786:role/myRedshiftRole


In [12]:
try:
    response = redshift.create_cluster(
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),
        
        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword =DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles = [roleArn]
    )
except Exception as e:
    print(e)

An error occurred (InvalidParameterValue) when calling the CreateCluster operation: The parameter MasterUserPassword must contain at least 1 upper case letter.


### 2.1 Describe the cluster to see its status

In [9]:
def prettyRedshiftProps(props):
    pd.set_options('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberofNodes", "VpcId"]
    x = [(k,v) for k, v in props.items() if k in keysToShow]
    return pd.DataFRame(data=x, columsn=["Keys", "values"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier = DWH_CLSUTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

NameError: name 'DWH_CLSUTER_IDENTIFIER' is not defined

### 2.2 Take note of the cluster endpoint and role ARN

<font color='red'>DO NOT RUN THIS UNLESS the cluster status becomes "Available". Make ure you are checking your Amazon Redshift cluster in the **us-west-2** region.
</font>

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps["IamRoles"][0]['IamRoleArn']
print("DWH_ENDPOINT ::", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

### STEP 3: Open an incoming TCP port to acces the cluster endpoint

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName = defaultSg.group_name,
        CidrIp = '0.0.0.0/0',
        IpProtocol = 'TCP',
        FromPort = int(DWH_PORT),
        ToPort = int(DWH_PORT))
except Exception as e:
    print(e)

### STEP 4: Make Sure you can connect to the cluster

In [13]:
%load_ext sql

In [ ]:
conn_string  "psotgres://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
print(conn_string)
%sql $conn_string

## STEP 5: CLEAN UP YOUR RESOURCES

<b><font color = "red"> DO NOT RUN THIS UNLESS YOU ARE SURE <br/>

In [ ]:
#CAREFULL !!
#-- Uncomment & run to delete the created resources
#redshift.delete_cluster(CluterIdentifier  =DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)
#CAREFULL !!

> run this block several times until the cluster really deleted

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
#iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
#iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!

In [ ]:
%sql SELECT * FROM artists limit 10;

In [ ]:
%sql SELECT * FROM time limit 10;

In [ ]:
%sql SELECT * FROM songs limit 10;

In [ ]:
%sql SELECT * FROM songplays order by songplay_id limit 10;

In [ ]:
%sql
delte from staging_events;
delete from staging_songs;
delete from songplays;
delete from users;
delete from songs;
delete from artists;
delete from time;